### Creating a QFrame

The best way to create a QFrame from scratch is to first create a `store.jon` file using Store(). This way you can build an initial structure for your work.

You can create a `store.json` with the below code. Refer to the Store tutorial here on the left if you want to learn more about this functionality. For now we will focus on QFrame.

In [ ]:
from grizly import Store, get_path, QFrame, union
from grizly import get_path

json_path = get_path("acoe_projects","dev", "grizly",
                            "notebooks","store.json")
store = Store(json_path)
store.add_query(["col1", "col2"], "schema", "table", "query_xyz")
store.to_store()

**Great!** So now we have a `store.json` file created, we want to create a QFrame. Let's do that:

In [ ]:
qf = QFrame().read_json(json_path=json_path, subquery="query_xyz")

### Working with the QFrame

#### Getting the sql

In [1]:
qf.get_sql()

NameError: name 'qf' is not defined

#### Doing some basic SQL stuff

The above code should generate the query below. This is a very simple query but you can see that this was created from your `store.json` file.
```sql
SELECT col1,
       col2
FROM schema.table
```

Now let's add a `where` statement, a `limit` and a `calculated` field all in 1 go:

In [ ]:
qf.query("col2 = 'some col2 value'") #<- where
qf.limit(10)
qf.assign(calculated_field = "col2*2")
qf.get_sql()

The above code will generate the below sql. We have quickly generated a query with some of the most commonly used sql statements.

```sql
SELECT col1,
       col2,
       col2*2 AS calculated_field
FROM schema.table
WHERE col2 = 'some col2 value'
  AND col2 = 'some col2 value'
LIMIT 10
```

As we update the `qf` object we are updated the `qf.data` variable which is a normal python dictionary. Go ahead and check what happens if you do `print(qf.data)`.

#### Forking

Forking qframes can be important if your data workflow needs to take the same sql table and apply different transformations to it.

Sometimes we want to fork, do some transforms, then union the qframes back together which results into an append operation on the data side.

By the way the return value of `union()` is also a qframe and if you do `uqf.get_sql()` below you get

```sql
SELECT col1,
       col2
FROM schema.table
UNION ALL
SELECT col1,
       col2
FROM schema.table
```

In [ ]:
qf2 = qf.copy() #<- forking
uqf = union(qframes=[qf, qf2], union_type="UNION ALL")

### Joining data

Unioning data is easy and described in the paragraph above. Joining data is equally easy, but we need to pay attention to the `on` argument.

First off, here is the SQL code generated by the below python code:

```sql
SELECT sq1.col1 AS col1,
       sq1.col2 AS col2
FROM
  (SELECT col1,
          col2
   FROM schema.table) sq1 INNER
  (SELECT col1,
          col2
   FROM schema.table) sq2 ON 'sq1.col1' = 'sq2.col1'
```

As you can see the expression after `ON` is exactly the same as the expression the user puts in the `on` value of the `join()` argument. It is key to remember to add `sq1` for the first qframe in `qframes` and `sq2` for the second



In [ ]:
from grizly import join

jq = join(qframes=[qf, qf2], join_type="INNER", on="'sq1.col1' = 'sq2.col1'")

jq.get_sql()

### Using the store.json

We use a `.json` file to conviniently manipulate information about columns, renames and other things that might be very verbose to manipulate in python code. We can edit the json file into a json editor like http://jsoneditoronline.org/ more conviniently than in Python code.

After editing the `store.json` we can read it back inside a QFrame using `read_json()`.

This means we can use our json as our main `store` of verbose information and python as our main way to manipulate said information.

This is how the structure of an sql query looks like in `store.js`:

```json
"sales_flash": {
    "select": {
    "fields": {
        "dimension_column_1": {
        "type": "dim",
        "group_by": "group"
        },
        "value_column_1": {
        "type": "dim",
        "as": "some as name",
        "group_by": "sum"
        }
    },
    "schema": "schema",
    "where": "dimension_column_1='abc'"
    "table": "table",
    "limit": 10
    }
}
```

We have there:

* The key or subquery name `sales_flash`
* The `select`, here we have all the information to generate the sql
* The `select.fields` all field type information
* The `schema`, the `table`, the `limit` and the `where`

### In database workflow

Sometimes all the data is in 1 database. Like for instance it is all in Redshift.

In this case we want to materialize the new transformed data inside a new Redshift table. This will make our workflow a lot faster as the data will not need to come back to our desktop and then uploaded again into the database.

We need to keep in mind a couple of things. First, we need to make sure to run to_sql() on the qframe, this way grizly will generate the internal sql necessary to create the table. Second, we need to create the table. Finally we can upload the qframe sql into our new table. Note, if the table already exists we don’t need to run get_sql or create_table.

```python
from grizly import set_cwd, QFrame
json = set_cwd("acoe_projects", "training", "subquery.json")
qf = QFrame(engine="mssql+pyodbc://Redshift").read_json(json_path = json, subquery="sandbox")
qf.get_sql()
qf.create_table("testing2", schema="z_sandbox_ac")
qf.to_table(table = "testing2", schema = "z_sandbox_ac", if_exists="replace")
```